In [1]:
import numpy as np
import pandas as pd

# Task 0
Read the dataset from csv file & perform data cleaning - remove all rows, which contains `?` in some columns.
Also check for data correctness (salary & salary $K).

In [10]:
df = pd.read_csv("../data/adult.csv")
df.replace("?", np.nan)
df.dropna(inplace=True)
df["Data correctness"] = df.apply(
    lambda row: (row["salary"] == '<=50K' and 0 <= row["salary K$"] <= 50) or
                (row["salary"] == '>50K' and row["salary K$"] > 50),
    axis=1
)
df

,Unnamed: 0,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,salary,salary K$,valid_salary
0,0,39,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,40,United-States,<=50K,39,True
1,1,50,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,13,United-States,<=50K,35,True
2,2,38,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,40,United-States,<=50K,27,True
3,3,53,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,40,United-States,<=50K,43,True
4,4,28,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,40,Cuba,<=50K,25,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,32556,27,Private,Assoc-acdm,Married-civ-spouse,Tech-support,Wife,White,Female,38,United-States,<=50K,36,True
32557,32557,40,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,40,United-States,>50K,173,True
32558,32558,58,Private,HS-grad,Widowed,Adm-clerical,Unmarried,White,Female,40,United-States,<=50K,40,True
32559,32559,22,Private,HS-grad,Never-married,Adm-clerical,Own-child,White,Male,20,United-States,<=50K,38,True


# Task 1
Print the count of men and women in the dataset.

In [15]:
df["sex"].value_counts()

sex
Male      21790
Female    10771
Name: count, dtype: int64

# Task 2
Find the average age of men in dataset

In [20]:
avg_men_age = df[df["sex"] == "Male"]["age"].mean()
avg_men_age

np.float64(39.43354749885268)

# Task 3
Get the percentage of people from Poland (native-country)

In [31]:
percen_poland_people = (df[df["native-country"] == "Poland"]["native-country"].value_counts())/len(df) * 100
percen_poland_people

native-country
Poland    0.18427
Name: count, dtype: float64

# Task 4
Get the mean and standard deviation of the age for people who earn > 50K per year. After this, get it for those who earn <= 50K.

In [40]:
mean_earn_more_50k = df[df["salary"] == ">50K"]["age"].mean()
std_earn_more_50k = df[df["salary"] == ">50K"]["age"].std()
mean_earn_less_50k = df[df["salary"] == "<=50K"]["age"].mean()
std_earn_less_50k = df[df["salary"] == "<=50K"]["age"].std()
mean_earn_more_50k, std_earn_more_50k, mean_earn_less_50k, std_earn_less_50k


(np.float64(44.24984058155847),
 np.float64(10.519027719851826),
 np.float64(36.78373786407767),
 np.float64(14.02008849082488))

# Task 5
Check, if there are some people without higher education (education: Bachelors, Prof-school, Assoc-acdm, Assoc-voc, Masters, Doctorate), but with > 50K salary

In [41]:
df[~df["education"].isin(["Bachelors", "Prof-school", "Assoc-acdm", "Assoc-voc", "Masters", "Doctorate"]) & (df["salary"] == ">50K")]

,Unnamed: 0,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,salary,salary K$,valid_salary
7,7,52,Self-emp-not-inc,HS-grad,Married-civ-spouse,Exec-managerial,Husband,White,Male,45,United-States,>50K,307,True
10,10,37,Private,Some-college,Married-civ-spouse,Exec-managerial,Husband,Black,Male,80,United-States,>50K,116,True
27,27,54,?,Some-college,Married-civ-spouse,?,Husband,Asian-Pac-Islander,Male,60,South,>50K,275,True
38,38,31,Private,Some-college,Married-civ-spouse,Sales,Husband,White,Male,38,?,>50K,166,True
55,55,43,Private,Some-college,Married-civ-spouse,Tech-support,Husband,White,Male,40,United-States,>50K,341,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32510,32510,39,Private,HS-grad,Married-civ-spouse,Prof-specialty,Husband,White,Male,45,?,>50K,212,True
32518,32518,57,Local-gov,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,40,United-States,>50K,116,True
32519,32519,46,Private,Some-college,Married-civ-spouse,Exec-managerial,Husband,White,Male,48,United-States,>50K,239,True
32557,32557,40,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,40,United-States,>50K,173,True


# Task 6
Get the statistics of age for each type of education. Use `groupby` and `describe` for this.

In [43]:
df.groupby("education")["age"].describe()

,count,mean,std,min,25%,50%,75%,max
education,,,,,,,,
10th,933.0,37.429796,16.720713,17.0,22.00,34.0,52.0,90.0
11th,1175.0,32.355745,15.545485,17.0,18.00,28.0,43.0,90.0
12th,433.0,32.000000,14.334625,17.0,19.00,28.0,41.0,79.0
1st-4th,168.0,46.142857,15.615625,19.0,33.00,46.0,57.0,90.0
5th-6th,333.0,42.885886,15.557285,17.0,29.00,42.0,54.0,84.0
7th-8th,646.0,48.445820,16.092350,17.0,34.25,50.0,61.0,90.0
9th,514.0,41.060311,15.946862,17.0,28.00,39.0,54.0,90.0
Assoc-acdm,1067.0,37.381443,11.095177,19.0,29.00,36.0,44.0,90.0
Assoc-voc,1382.0,38.553546,11.631300,19.0,30.00,37.0,46.0,84.0


# Task 7
Compare the married and non-married men salaries. Who earns more? (>50K or <=50K)
Married men are those, whom `marital-status` starts with "Married". Others are not.

In [49]:
df["is_married"] = df["marital-status"].str.startswith("Married")
salary_comparison = df[df["sex"] == "Male"].groupby(["is_married", "salary"])["salary"].count()
salary_comparison

is_married  salary
False       <=50K     7552
            >50K       697
True        <=50K     7576
            >50K      5965
Name: salary, dtype: int64

# Task 8
Get the max hours per week some person works. How many people works the same amount of hours per week?

In [54]:
max_hours = df["hours-per-week"].max()
count_people = df[df["hours-per-week"] == max_hours].shape[0]
print(f"{count_people} people works {max_hours} hours")

85 people works 99 hours


# Task 9
Analyze the correlation between data in dataset. Understand connected fields in it and print highlight thier connection.

In [57]:
dummies = pd.get_dummies(df)
correlation_matrix = dummies.corr()
correlation_matrix

,Unnamed: 0,age,hours-per-week,salary K$,valid_salary,is_married,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,...,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia,salary_<=50K,salary_>50K
Unnamed: 0,1.000000,0.001286,0.000607,-0.001666,NaN,0.004782,-0.003062,0.000068,-0.000542,0.005617,...,0.004146,-0.005129,-0.004399,-0.001624,0.005810,0.001983,0.000401,0.004196,-0.005404,0.005404
age,0.001286,1.000000,0.068756,0.201774,NaN,0.318239,0.042627,0.051227,0.060901,-0.019362,...,0.002583,0.000613,-0.013816,-0.006271,0.004843,0.015917,-0.015054,0.000375,-0.234037,0.234037
hours-per-week,0.000607,0.068756,1.000000,0.196916,NaN,0.210912,-0.168640,0.013293,0.011576,-0.014262,...,0.001264,0.007938,-0.004988,0.009537,-0.006006,0.002446,-0.011379,0.007408,-0.229689,0.229689
salary K$,-0.001666,0.201774,0.196916,1.000000,NaN,0.374539,-0.064147,0.052179,0.028871,-0.007494,...,0.000567,-0.004280,0.012730,-0.001056,-0.003691,0.028293,-0.014626,0.008678,-0.855788,0.855788
valid_salary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
native-country_United-States,0.001983,0.015917,0.002446,0.028293,NaN,-0.033716,0.006193,0.015441,0.033196,0.005000,...,-0.056315,-0.145558,-0.116167,-0.068978,-0.070869,1.000000,-0.133180,-0.065031,-0.034470,0.034470
native-country_Vietnam,0.000401,-0.015054,-0.011379,-0.014626,NaN,-0.005054,-0.002286,0.000099,-0.003611,-0.000666,...,-0.000872,-0.002254,-0.001799,-0.001068,-0.001097,-0.133180,1.000000,-0.001007,0.017649,-0.017649
native-country_Yugoslavia,0.004196,0.000375,0.007408,0.008678,NaN,0.012280,-0.005420,-0.003865,-0.000161,-0.000325,...,-0.000426,-0.001100,-0.000878,-0.000521,-0.000536,-0.065031,-0.001007,1.000000,-0.006959,0.006959
salary_<=50K,-0.005404,-0.234037,-0.229689,-0.855788,NaN,-0.434944,0.078198,-0.059372,-0.033091,0.008259,...,-0.000413,0.004737,-0.014020,0.004078,0.007660,-0.034470,0.017649,-0.006959,1.000000,-1.000000
